In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
def request_song_info(song_title, artist_name):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + 'CQ4Ni_6zhB_MBUMbHmxQ6OlFJjwrZqNW1hhuH7SGS1rxP-CM8StFqZ9en4N0Iaap'}
    search_url = base_url + '/search'
    data = {'q': song_title + ' ' + artist_name}
    response = requests.get(search_url, data=data, headers=headers)

    return response

    
def scrap_song_url(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()

    return lyrics

In [10]:
response = request_song_info('dkr','booba')
print(response)

<Response [200]>


In [11]:
song_data = pd.read_csv('Corpus_file.csv') 
song_data.info()
print(song_data.columns.tolist())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 5 columns):
Artist         334 non-null object
Title          331 non-null object
Style          331 non-null object
Duration       178 non-null object
lien_lyrics    191 non-null object
dtypes: object(5)
memory usage: 13.2+ KB
['Artist', 'Title', 'Style', 'Duration', 'lien_lyrics']


In [12]:
song_style = song_data["Style"]
song_time = song_data["Duration"]

In [13]:
nb_song = 40
song_title = song_data["Title"].tolist()[:nb_song]
artist_name = song_data["Artist"].tolist()[:nb_song]
song_style = song_data["Style"].tolist()[:nb_song]
song_time = song_data["Duration"].tolist()[:nb_song]
print(song_title)

['Paris-Seychelles', 'Le chant des sirènes', 'Jour 1', "Les filles d'aujourd'hui", 'Color Gitano', 'Dernière Danse', "L'attrape-rêves", 'A bouche que veux-tu', 'Paradis secret', 'Un Jour Au Mauvais Endroit', "Rien qu'une fois", 'Je suis chez moi', 'Sapés comme jamais (Pilule Bleue)', 'Oui ou non', 'VIDEOCLUB', 'Ne m’invite pas', 'Défait', 'Casanova', 'Maria Maria', 'Les matins', 'Clown', 'En feu', 'Zaho', 'Souf', 'Alors on danse', 'Papaoutai', 'tous les mêmes', 'Basique', 'Alors souris', 'Tout le monde est loin d’être fiable', 'Ma beauté (Pilule Violette)', 'Dommage', 'Balance Ton Quoi', 'Bim Bam toi', 'Bella', 'J’me tire', 'Jaloux', 'Reine', 'Les gens', "On S'Attache"]


In [14]:
#song_title = ['dkr','champs elysee']
#artist_name = ['booba','sch']
x = []
for i in range(len(song_title)):
    response = request_song_info(song_title[i], artist_name[i])
    json = response.json()
    remote_song_info = None

    for hit in json['response']['hits']:
        if artist_name[i].lower() in hit['result']['primary_artist']['name'].lower():
            remote_song_info = hit
            break

    if remote_song_info:
            song_url = remote_song_info['result']['url']
            x.append(scrap_song_url(song_url))

In [16]:
# Deliminiter strophe
indice_strophe = []
for i in range(len(x)):
    res = []
    test_sub = "\n\n["
    test_str= x[i]
    res =[i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]
    indice_strophe.append(res)

In [18]:
#Delimiteur nom strophe
test_sub = "["
test_str= x[0]
res2 = []
res2 = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]

test_sub = "]"
res3 = []
res3 = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]
print(res2)
print(res3)

[122, 278, 386, 543, 607, 754, 893, 909]
[132, 285, 396, 551, 617, 761, 901, 916]


In [19]:
def getmetadata(test_str):
    liste = []
    test_sub = "["
    test_str = test_str
    res2 = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]
    test_sub = "]"
    res3 = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]
    liste.append(res2)
    liste.append(res3)
    return liste

In [20]:
indice_metadata = []
for i in range(len(x)):
    indice_metadata.append(getmetadata(x[i]))
    

In [21]:
s = x
strophe_name = []
for i in range(len(indice_metadata)):
    name = []
    for k in range(len(indice_metadata[i][0])):
        start = indice_metadata[i][0][k]  
        end = indice_metadata[i][1][k] 
        name.append(s[i][start+1:end])
    strophe_name.append(name)

print(strophe_name)

[['Couplet 1', 'Pont 1', 'Couplet 2', 'Refrain', 'Couplet 3', 'Pont 2', 'Refrain', 'Pont 3'], ['Couplet 1', 'Pre-Refrain', 'Refrain', 'Couplet 2', 'Pre-Refrain', 'Refrain', 'Refrain', 'Outro'], ['Couplet 1', 'Refrain', 'Couplet 2', 'Refrain'], ['Couplet 1', 'Pré-refrain', 'Refrain', 'Couplet 2', 'Pré-refrain', 'Refrain', 'Couplet 3', 'Refrain'], ['Couplet 1', 'Refrain', 'Pont 1', 'Refrain', 'Pont 2', 'Refrain'], ['Couplet 1', 'Refrain', 'Couplet 2', 'Refrain', 'Couplet 3'], ['Couplet 1', 'Refrain', 'Couplet 1', 'Refrain', 'Couplet 2', 'Refrain', 'Outro'], ['Couplet 1', 'Refrain', 'Couplet 2', 'Refrain', 'Pont', 'Refrain'], ['Couplet 1', 'Refrain', 'Couplet 2', 'Refrain', 'Pont', 'Outro'], ['Couplet 1', 'Refrain', 'Couplet 2', 'Refrain', 'Pont', 'Refrain', 'Outro'], ['Couplet 1', 'Pré-refrain', 'Refrain', 'Couplet 2', 'Pré-refrain', 'Refrain', 'Pont', 'Refrain', 'Outro'], ['Couplet 1', 'Refrain', 'Couplet 2', 'Refrain', 'Pont', 'Refrain', 'Post-refrain', 'Outro'], ['Couplet 1', 'Pré-Ref

In [22]:
s = x
strophe_clean = []
m = []
for i in range(len(indice_strophe)):
    music = []
    print("######################")
    for j in range(len(indice_strophe[i])-1):
        start = indice_strophe[i][j]  
        end = indice_strophe[i][j+1]
        print("Limite :",start,end)
        data = s[i]
        data_clean = data[start+2:end]
   
        print(i,j)
        data_clean2 = data_clean.split("["+strophe_name[i][j]+"]")
        if len(data_clean2) > 1:
            data_clean2 = data_clean.split("["+strophe_name[i][j]+"]")[1]
        print("TEXTE : " , data_clean2)
        print(strophe_name[i][j])
        music.append(data_clean2)
        print("------------")
        
    strophe_clean.append(music)
    



######################
Limite : 120 276
0 0
TEXTE :  
Te sourire dehors à Angoulême
Un chasse-spleen Melchior Paris-Seychelles
Découvrir ton corps depuis la scène
Du sombre décor Lucrèce est reine
Couplet 1
------------
Limite : 276 384
0 1
TEXTE :  
On s'était dit des choses
Que l'on ne tiendra pas
Le temps que l'eau se pose
Sur nos ronces lilas
Pont 1
------------
Limite : 384 541
0 2
TEXTE :  
Le soleil s'endort sur Seychelles
Le sable et l'aurore, fleur de sel
Attends-moi je sors, de l'Île aux cerfs
L'orage est de l'or pour se perdre
Couplet 2
------------
Limite : 541 605
0 3
TEXTE :   (x4)
I need you so, oh oh
I won't let you go, woh oh
Refrain
------------
Limite : 605 752
0 4
TEXTE :  
Souffrir la mort pour te plaire
Défendre nos torts et puis se taire
Le désir encore pour l'hiver
Un chaos d'efforts sur Saint Hilaire
Couplet 3
------------
Limite : 752 891
0 5
TEXTE :  
On s'était dit des choses
Que l'on ne tiendra pas
Le temps que l'on se pose
Sur nos lauriers mayas

On s'étai

In [26]:
"""test_sub = "\n"
for i in range(len(substring)):
    test_str= substring[i]
    r = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]
    res4.append(r)
print(res4)"""

'test_sub = "\n"\nfor i in range(len(substring)):\n    test_str= substring[i]\n    r = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)]\n    res4.append(r)\nprint(res4)'

In [ ]:
import xml.etree.ElementTree as ET
import xml.etree.cElementTree as ET

In [ ]:

root = ET.Element("root")
for i in range(len(strophe_clean)):
    doc = ET.SubElement(root, "doc",name =song_title[i],artist =artist_name[i],style=song_style[i],duree=song_time[i],genre="")
    for j in range(len(strophe_clean[i])-1):
        ET.SubElement(doc, "lg",name=strophe_name[i][j], type="strophe").text = strophe_clean[i][j]

tree = ET.ElementTree(root)
tree.write("Lyric_song.xml", encoding="UTF-8")

In [27]:

"""
import json

data = {}
data['music'] = []
for i in range(len(strophe_clean)):
    for j in range(len(strophe_clean[i])-1):
        data['music'].append({
            'artist': artist_name[i],
            'title' : song_title[i],
            'duree': song_time[i],
            'style': song_style[i],
            'lyric': strophe_clean[i][j]
        })


with open('data.json', 'w') as outfile:
    json.dump(data, outfile)
    """

"\nimport json\n\ndata = {}\ndata['music'] = []\nfor i in range(len(strophe_clean)):\n    for j in range(len(strophe_clean[i])-1):\n        data['music'].append({\n            'artist': artist_name[i],\n            'title' : song_title[i],\n            'duree': song_time[i],\n            'style': song_style[i],\n            'lyric': strophe_clean[i][j]\n        })\n\n\nwith open('data.json', 'w') as outfile:\n    json.dump(data, outfile)\n    "